# Push data to ES

## Major

In [1]:
import sys
sys.path.append("../")
from src.config import Configuration
from src.prepare import data_load

doc_loader = data_load.DocDataLoader()
config = Configuration()

In [2]:
db = doc_loader.load_major_docs(size = 460, overlap=20)

From (44, 4) to Counter({2: 37, 1: 6, 3: 1})


### Gemini

In [3]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

INDEX = "gemini-major"
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",
                                          google_api_key=config.load_gemini_token())

/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### VN SBert

In [3]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
INDEX = "vnsbert-major"

MODEL = "keepitreal/vietnamese-sbert"
embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=conf.load_hg_token(0),
    model_name=MODEL)

### LaBSE

In [3]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
INDEX = "labse-major"

MODEL = "sentence-transformers/LaBSE"

embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=config.load_hg_token(1),
    model_name=MODEL)

## Load

In [4]:
from langchain_community.vectorstores.elasticsearch import ElasticsearchStore

es = ElasticsearchStore.from_documents(db, embedding=embeddings,
                          es_connection=config.load_elasticsearch_connection(),
                          index_name=INDEX,
                          distance_strategy="EUCLIDEAN_DISTANCE")